In [6]:
pip install tweepy

     |████████████████████████████████| 153kB 3.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import os
import json
import os
import tweepy as tw
import pandas as pd
import datetime

In [1]:
consumer_key= '9a4nptu5V6lWvQuRvqfWxAMG3'
consumer_secret= '0zrkzfG3yx6Typj5yHq48glfrEyWAkzmjzai1OwNS1uBOeeOkN'
access_token= '1086150803851665408-72glgXOkwiJZvuMhUrbV1tx3PZkn24'
access_token_secret= '50mx14gNfqbE4cUqWVoJu2rWGrStMHWsG8NTqNqQPQrcM'


auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [2]:

tweet_table = []

In [5]:
search_words = ["feel", "feeling"]
for i in range(len(search_words)):
    new_word = search_words[i] + " -filter:retweets"
    tweets = tw.Cursor(api.search,
              q=new_word,
              lang="en").items()
    for tweet in tweets:
        if not tweet.retweeted:
            tweet_table.append(tweet.text)

            
tweet_text = pd.DataFrame(data=tweet_table, 
                    columns=["texts"])
    

TweepError: Failed to send request: ('Connection aborted.', OSError("(54, 'ECONNRESET')"))

In [14]:
print(len(tweet_table))
with open('/Users/niloofar/Desktop/last-seven-days-tweets.txt','w+') as file:
        json.dump(tweet_table, file);


183415


In [16]:
tweet_text = pd.DataFrame(data=tweet_table, 
                    columns=["texts"])

print(tweet_text.head())

                                               texts
0  @devinbennett__ @ShitronsWifesBF @NamelessOne0...
1  @tommy_probably @BenjiSales Benji post made yo...
2  hey!  this account is a safe and quiet place f...
3  @boogiemanMD @jahsoulgood @VALIANT_UTD Oh ur B...
4  @Ripple @MonicaLongSF At least one of the top ...


In [17]:
import torch
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer

In [12]:
#preprocessing tweeter data:
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import seaborn as sns
sns.set_style('darkgrid')


def format_text(texts, col):
    #Remove @ tags
    comp_df = texts.copy()
    
    # remove all the punctuation
    comp_df[col] = comp_df[col].str.replace(r'(@\w*)','')

    #Remove URL
    comp_df[col] = comp_df[col].str.replace(r"http\S+", "")

    #Remove # tag and the following words
    comp_df[col] = comp_df[col].str.replace(r'#\w+',"")

    #Remove all non-character
    comp_df[col] = comp_df[col].str.replace(r"[^a-zA-Z ]","")

    # Remove extra space
    comp_df[col] = comp_df[col].str.replace(r'( +)'," ")
    comp_df[col] = comp_df[col].str.strip()

    # Change to lowercase
    comp_df[col] = comp_df[col].str.lower()
    
    return comp_df

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niloofar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/niloofar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
formated_tweets = format_text(tweet_text,'texts')

# Drop the columns we don't want
print(formated_tweets.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niloofar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/niloofar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(183415, 1)


In [20]:
formated_tweets.head()

,texts
0,i went through the same thing and im
1,benji post made you feel
2,hey this account is a safe and quiet place for...
3,oh ur brazilian okay i feel you lol weve alway...
4,at least one of the top women investors im exc...


In [22]:
text = formated_tweets.values.tolist()

In [23]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("Using device:", device)

def get_sentiment(model, tokenizer, text_batch): #Large batches cause memory problems
    model.eval()
    with torch.no_grad():
        encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True).to(device)
        outputs = model(encoding['input_ids']).logits
        pred = torch.argmax(outputs, axis=1)
    return pred.detach().numpy()

def main():
    model = BertForSequenceClassification.from_pretrained('./sent_0.pt').to(device)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    negative = 0
    positive = 0
    neutral = 0
  #  text = formated_tweets.values.tolist()
    for i in range(len(text)):
        pred = get_sentiment(model, tokenizer, text[i])
        v = {0: "negative", 1: "neutral", 2:"positive"}
        if pred == 0:
            negative +=1
        elif pred == 1:
            neutral +=1
        else:
            positive +=1
    print("negative: ", negative, "positive: ", positive, "neutral: ", neutral)
if(__name__ == "__main__"):
    main()

Using device: cpu
negative:  116255 positive:  67160 neutral:  0


In [152]:
#Test!

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("Using device:", device)

def get_sentiment(model, tokenizer, text_batch): #Large batches cause memory problems
    model.eval()
    with torch.no_grad():
        encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True).to(device)
        outputs = model(encoding['input_ids']).logits
        pred = torch.argmax(outputs, axis=1)
    return pred.detach().numpy()

def main():
    model = BertForSequenceClassification.from_pretrained('./sent_0.pt').to(device)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    text = ["Have a fantastic day!", "Life is awful. I feel depressed."]
    pred = get_sentiment(model, tokenizer, text)

    v = {0: "negative", 1: "neutral", 2:"positive"}
    print([v[x] for x in pred])

if(__name__ == "__main__"):
    main()

Using device: cpu
['positive', 'negative']
